# Inputs

In [1]:
# Path to chromedriver. Use same \ and keep leading r
chromedriver_path = r"C:\Users\Alessandro Bitetto\Downloads\UniPV\Scraping_PEC\WebDriver\chromedriver"

# web page
web_page = "https://www.ufficiocamerale.it/cerca-pec-azienda"

# VAT list  - VAT must be without leading "IT" - .csv with 1 column "VAT" - Use same \ and keep leading r
VAT_file = r"C:\Users\Alessandro Bitetto\Downloads\110Laude\110L_SIR\03_Output\df_VAR_for_PEC.csv"
VAT_file_sep = ";"

# output csv separator
OUT_file_sep = ";"

# Scrape

In [2]:
from utils import get_chromedriver, scrape_PEC

scrape_PEC(chromedriver_path, web_page, VAT_file, VAT_file_sep, OUT_file_sep)

Launching Chrome...OK



ElementClickInterceptedException: Message: element click intercepted: Element <input type="text" class="form-control" id="partita_iva" placeholder="  "> is not clickable at point (663, 354). Other element would receive the click: <div class="c-line lightgrey-bg-c1 px-4 py-4">...</div>
  (Session info: chrome=101.0.4951.67)


In [3]:
import os
import pandas as pd
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import time, re
from timeit import default_timer as timer
import datetime

from utils import get_chromedriver, scrape_PEC

In [4]:
LOG_FILE = "log.txt"
SMALL_TIME_SLEEP = 0.5
start = timer()

# read VAT list
df_VAT = pd.read_csv(VAT_file, dtype = 'str', sep = VAT_file_sep)

# create/reload log
if os.path.exists(LOG_FILE):
    log = pd.read_table(LOG_FILE, delimiter = ';', dtype = 'str')
else:
    log = pd.DataFrame(columns = ['VAT', 'PEC'])
    with open(LOG_FILE, 'a') as f:
        f.write('"VAT";"PEC"')

# open browser and go to website
print("Launching Chrome...", end ="")
driver = get_chromedriver(chromedriver_path = chromedriver_path)
driver.implicitly_wait(3)
driver.get(web_page)
print("OK", end ="\n\n")

Launching Chrome...OK



In [5]:
vat = "03091390546"

In [ ]:
# loop VAT
df_out = pd.DataFrame(columns = ['VAT', 'PEC'])
tot_vat = df_VAT.shape[0]
for vat_i, vat in enumerate(df_VAT.VAT.values):

    print("Querying VAT", str(vat_i+1), "/", str(tot_vat), end = "\r")
    if vat not in log.VAT.values:    # check if VAT already in log
        stop = 0
        while stop == 0:
            driver.find_element_by_id("partita_iva").click()
            driver.find_element_by_id("partita_iva").clear()
            driver.find_element_by_id("partita_iva").send_keys(vat)
            driver.find_element_by_id("button_calcola").click()
            time.sleep(SMALL_TIME_SLEEP)
            pec = driver.find_element_by_id('codice').get_attribute("textContent")
            if pec == '':
                pec = 'not found'
            if pec != 'not found' and pec not in df_out.PEC.values:
                stop = 1

        # update and save log
        log = log.append({'VAT': vat, 'PEC': pec}, ignore_index=True)
        with open(LOG_FILE, 'a') as f:
            f.write('\n"'+vat+'";"'+pec+'"')
    else:
        pec = log[log.VAT == vat].PEC.values[0]

    # update list
    df_out = df_out.append({'VAT': vat, 'PEC': pec}, ignore_index=True)

# check for duplicated PEC
check_PEC = df_out.PEC.value_counts().rename_axis('PEC').reset_index(name='counts')
if check_PEC.counts.max() > 1:
    duplicated_PEC = check_PEC[check_PEC.counts > 1]
    duplicated_PEC.merge(df_out, on='PEC').sort_values(by=['PEC']).to_csv('duplicated_PEC.csv', sep=OUT_file_sep, index=False, quoting=1)
    print('\n\n ###### duplicated PEC found:', duplicated_PEC.shape[0], ' - saved in "duplicated_PEC.csv"\n')

# display report of missing PEC
print('\n\nVAT without PEC:', df_out[df_out.PEC == "not found"].shape[0])

# save output
df_out.to_csv('PEC_list.csv', sep=OUT_file_sep, index=False, quoting=1)